## コンテンツベースのフィルターのデータセットを作成する

このノートブックは、コンテンツベースのモデルの作成に使用するデータを作成します。 BigQueryで公開されているKurier.atデータセットからSQLクエリのコレクションを介してデータを収集します。
Kurier.atはオーストリアの新しいサイトです。これらのラボの目標は、サイトへの訪問者に記事を推奨することです。このラボでは、トレーニング用のデータを収集し、後続のノートブックでレコメンダーモデルをトレーニングします。

このノートブックは
* BigQueryテーブルからデータをプルしてローカルファイルに書き込む方法
*再現可能なトレインとテストの分割を行う方法

In [1]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

PROJECT = 'qwiklabs-gcp-00-c6695d766645' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-00-c6695d766645' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [2]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


このヘルパー関数を使用して、データベース内の各記事の記事ID、カテゴリ、および作成者を含むリストをローカルファイルに書き込みます

In [3]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(line)

### BigQueryからデータをプルする

下のセルは、データセット内のすべての記事ID（つまり、「コンテンツID」）を含むローカルテキストファイルを作成します。

[BigQuery]（https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample）の元のデータセットをご覧ください。次に、以下のクエリを読み、それが何をしているのかを理解していることを確認してください。

In [5]:
sql="""
#standardSQL

SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
    
"""

content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("記事の合計数は {}".format(len(content_ids_list)))

Some sample content IDs ['299866366', '299865757', '299824032']
記事の合計数は 15634


データベースには15,634件の記事があるはずです。
次に、記事のカテゴリのリストと記事の作成者のリストを含むローカルファイルを作成します。

記事のカテゴリまたは著者情報を取得するときは、インデックスの変更に注意してください。また、記事の最初の著者を使用して著者リストを作成しています。
元のデータセットに戻って参照し、 `hits.customDimensions.index`フィールドを使用して正しいインデックスを確認します。

In [6]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['Lifestyle', 'News', 'Stars & Kultur']


カテゴリは、「ニュース」、「スターと文化」、「ライフスタイル」です。
著者リストを作成するときは、各記事の最初の著者情報のみを使用します。

In [7]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Moritz Gottsauner-Wolf', 'Georg Leyrer', 'Julia Schrenk', 'Peter Temel', 'Raffaela Lindorfer', 'Margaretha Kopeinig', 'Wolfgang Atzenhofer', 'Michaela Reibenwein', 'Elisabeth Holzer', 'Christian Willim']
The total number of authors is 385


データベースには385人の著者がいるはずです。

### トレインセットとテストセットを作成します。

このセクションでは、モデルをトレーニングするためのデータのトレーニング/テスト分割を作成します。訪問者IDとコンテンツIDの連結値を使用して、ファームフィンガープリントを作成し、トレーニングセットのデータの約90％とテストセットのデータの10％を取得します。

In [8]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) < 9
"""
training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1028781454955450337,299986548,News,Nachrichtenagentur: Nordkorea feuerte erneut R...,Stefan Berndl,574,299986548
1,1076604330971416042,299835130,News,André Heller holt Gandhi nach Tirol,Christian Willim,574,299777664
2,1096315653917377549,299575786,Lifestyle,Familie in Social-Media: Vier Millionen Klicks...,Daniela Davidovits,574,299012533
3,1096315653917377549,299012533,Stars & Kultur,"""Borat""-Erfinder Cohen will für Fans Strafe za...",Elisabeth Mittendorfer,574,299800661
4,1096315653917377549,299800661,Stars & Kultur,Meghan Markle im Porträt - Vom TV-Sternchen zu...,Christina Michlits,574,299821418


In [9]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) >= 9
"""
test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1138388341807275290,295177956,Lifestyle,Rostock im Sommer 2018 ab Linz,None,573,299824032
1,1145301083080858057,299828023,News,Glyphosat geht in die Verlängerung,Andreas Anzenberger,574,299814842
2,1160160574707650958,299918278,News,Skipässe in Wintersport-Hochburgen massiv teurer,Stefan Hofer,574,299899396
3,1194756252802104671,161933022,Lifestyle,6 geniale Beauty-Rituale für die Nacht,Maria Zelenko,550,299789055
4,1375185952017551641,299804373,Lifestyle,Bloggerin wegen Anstiftung zur Magersucht ange...,Elisabeth Mittendorfer,574,299697224


トレーニングセットとテストセットを含む、作成した2つのcsvファイルを見てみましょう。また、両方のファイルの行数をカウントして、約90/10のトレイン/テスト分割が達成されたことを確認します。
**コンテンツベースのフィルタリング**
次のノートブックでは、カテゴリ、タイトル、作成者、公開日など、現在読んでいる記事に関する情報を指定して、記事を推奨するモデルを構築します。

In [10]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [11]:
!head *_set.csv

==> test_set.csv <==
1138388341807275290,295177956,Lifestyle,Rostock im Sommer 2018 ab Linz,,573,299824032
1145301083080858057,299828023,News,Glyphosat geht in die Verlängerung,Andreas Anzenberger,574,299814842
1160160574707650958,299918278,News,Skipässe in Wintersport-Hochburgen massiv teurer,Stefan Hofer,574,299899396
1194756252802104671,161933022,Lifestyle,6 geniale Beauty-Rituale für die Nacht,Maria Zelenko,550,299789055
1375185952017551641,299804373,Lifestyle,Bloggerin wegen Anstiftung zur Magersucht angezeigt,Elisabeth Mittendorfer,574,299697224
1410187362777626442,299912041,News,Deutscher Bürgermeister bei Messerangriff schwer verletzt,,574,299912151
1410187362777626442,299912151,News,NÖ: Beißender Geruch in der Klasse,Jürgen Zahrl,574,299913879
1410187362777626442,299912085,News,Erster ÖBB-Containerzug nach China unterwegs,Stefan Hofer,574,299800704
1410187362777626442,293997546,News,Arbeiterkammer: Henry am Zug soll Strafe ausgefasst haben,Kid Möchel,573,299837992
141018736277